<a href="https://colab.research.google.com/github/Alejom74/Business-Inteligence/blob/main/Reto_2_Alejandro_Mariaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se procede con la carga de las librerias de trabajo

In [ ]:
import numpy as np
import pandas as pd

#Librerías del árbol
from  sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Se procede con la carga de los datos

In [ ]:
nxl ="/content/2. CollegeAdmisions_Int_M.xlsx"
XDB = pd.read_excel(nxl, sheet_name=0)
XDB=XDB.dropna()
XDB.head(100)

# Convertir la variable Gender a numérica (F:0, M:1)
XDB["Gender"] = XDB["Gender"].map({"F": 0, "M": 1})

# Seleccionar las variables
XD = XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd = XDB[['Admitted']]

1.1. Se determinan el número de personas que fueron admitidas a la Universidad

In [ ]:
# Convertir 'Yes' a 1 y 'No' a 0
XDB["Admitted"] = XDB["Admitted"].map({"Yes": 1, "No": 0})

# Ahora sí, contar los admitidos
num_admitidos = XDB["Admitted"].sum()
print(f"Número de personas admitidas: {num_admitidos}")

Número de personas admitidas: 1303


**Descripción de los datos**

De acuerdo con la base de datos de admisión universitaria, esta posee un total de 1528 registros de estudiantes, divididos en 1303 admitidos y 225 no admitidos. Cada uno de los solicitantes de admisión está descrito en función de 7 variables socioeconómicas:

**Edu_Parent1:** Nivel educativo alcanzado por el primer padre o tutor
**Edu_Parent2:** Nivel educativo alcanzado por el segundo padre o tutor.
**Gender:** Género del estudiante, representado numéricamente como 0 = Femenino, 1 = Masculino.
**White:** Variable categórica que indica si el estudiante se identifica como blanco (1 = Sí, 0 = No).
**Asian:** Variable categórica que indica si el estudiante se identifica como asiático (1 = Sí, 0 = No).
**HSGPA:** Promedio de calificaciones del estudiante en la escuela secundaria.
**SAT/ACT:** Puntaje obtenido en los exámenes estandarizados SAT o ACT.
**Admitted:** Indica si el estudiante fue admitido en la universidad (1 = Sí, 0 = No).
El análisis se centrará en estas variables para predecir la probabilidad de admisión mediante un modelo basado en árboles de decisión.

2. Se procede con implementacion del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4)
mar.fit(XD,yd)

#Se procede con la gráfica
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

ve=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']
dot_data=export_graphviz(mar, feature_names=ve)
graph=graph_from_dot_data(dot_data)
graph.write_png('Reto 2 Alejandro Mariaca.png')

#SE DETERMINA EL NUMERO DE NODOS PUROS PARA EL ARBOL ASOCIADO CON LA ESTRUCTURA DEL MODELO
num_nodos_puros = sum(mar.tree_.impurity == 0)
print(f"Número de nodos puros en el árbol (Gini = 0.0): {num_nodos_puros}")

Número de nodos puros en el árbol (Gini = 0.0): 7


3. Se procede con el pronostico de los solicitantes que no fueron admitidos

In [ ]:
# Cargar la hoja de pronóstico
XDB0 = pd.read_excel(nxl, sheet_name=1)

# Convertir Gender a numérico
XDB0["Gender"] = XDB0["Gender"].map({"F": 0, "M": 1})

# Crear la base de datos para pronóstico
XD0 = XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd0 = XDB0[['Admitted']]

# Mantener todos los solicitantes
filas = XDB0.index

# Hacer la predicción
y_pred = mar.predict(XD0)

# Crear el informe en Excel
df = pd.DataFrame(XD0)
df["Admitted_Predicted"] = y_pred
df.insert(0, "Solicitante", filas)  # Agregar índice de solicitantes

# Guardar en Excel
df.to_excel("Pronostico Solicitantes No Admitidos.xlsx", index=False)

4. Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #Pronostico sobre la base de datos original
cm=confusion_matrix(yd,ydp)
print("La matriz de confusion es :\n",cm)

VN=cm[0,0]
FP=cm[0,1]
FN=cm[1,0]
VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN)
Sensibilidad=VP/(VP+FN)
Especificidad=VN/(VN+FP)
Precision=VP/(VP+FP)
TasaError=(FP+FN)/(VP+VN+FP+FN)
PredNeg=VN/(VN+FN)
PredPos=VP/(VP+FP)

print("La exactitud es: ",Exactitud)
print("La sensibilidad es: ",Sensibilidad)
print("La especificidad es: ",Especificidad)
print("La precision es: ",Precision)
print("La tasa de error es: ",TasaError)
print("La cantidad de predicciones negativas es: ",PredNeg)
print("La cantidad de predicciones positivas es: ",PredPos)

La matriz de confusion es :
 [[  20  205]
 [   2 1301]]
La exactitud es:  0.8645287958115183
La sensibilidad es:  0.9984650805832693
La especificidad es:  0.08888888888888889
La precision es:  0.8638778220451527
La tasa de error es:  0.1354712041884817
La cantidad de predicciones negativas es:  0.9090909090909091
La cantidad de predicciones positivas es:  0.8638778220451527


**Análisis de Resultados**

El modelo de árbol de decisión aplicado a la base de datos de admisión universitaria mostró un desempeño sólido en la clasificación de los solicitantes. La matriz de confusión refleja un alto número de verdaderos positivos (1301) y verdaderos negativos (20), mientras que los falsos positivos (205) y falsos negativos (2) son significativamente menores. Esto indica que el modelo tiene un buen desempeño en la identificación de estudiantes admitidos, aunque su capacidad para detectar correctamente a los no admitidos es baja. La exactitud del modelo es del 86.45%, lo que significa que en aproximadamente 8 de cada 10 casos, la predicción del modelo es correcta. Sin embargo, la tasa de error del 13.54% sugiere que aún hay margen de mejora. Un aspecto relevante es la sensibilidad, que alcanza un valor de 99.84%, demostrando que el modelo es altamente eficiente para predecir correctamente a los estudiantes admitidos. En contraste, la especificidad es de apenas 8.89%, lo que indica que la capacidad del modelo para detectar correctamente a los estudiantes no admitidos es baja. En cuanto a la precisión, se obtuvo un 86.39%, lo que confirma que el modelo es confiable al predecir la admisión de un estudiante. Sin embargo, la precisión negativa (90.90%) sugiere que, aunque pocos estudiantes son clasificados erróneamente como admitidos, la baja especificidad puede afectar su utilidad en escenarios donde es crucial identificar correctamente a los no admitidos. En conclusión, el modelo presenta un desempeño destacado en la clasificación de estudiantes admitidos, pero su baja especificidad indica que no distingue con la misma eficacia a los rechazados. Esto sugiere que podría mejorarse ajustando hiperparámetros o considerando técnicas de balanceo de datos para mejorar la detección de los no admitidos sin afectar la sensibilidad del modelo.